#### this notebook is for using scipy curve\_fit to tweak parameters in a known form

In [3]:
#from pysr import PySRRegressor, TemplateExpressionSpec

import matplotlib.pyplot as plt
import numpy as np

import camb

from scipy.optimize import curve_fit

In [4]:
# Load Data
pars     = np.load('CL_data/parameter_test10.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('CL_data/lensed_CL10.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('CL_data/unlensed_CL10.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [5]:
#extracting info for independent variables
ombs = pars[:,1]
omcs = pars[:,2]

ells = np.array([l for l in range(2, 5000)])

In [6]:
#getting dependent variables
lensing = lensed/unlensed #lensing tail
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])

In [7]:
#the fidicual values for normalizing variables
fid_ombh2 = 0.0224
fid_omch2 = 0.12

In [44]:
def func(X, b1):
    #expected form of the lensing tail function for curve fitting
    ell = X[0]
    ommh2 = X[1]
    lit_x = (ommh2/0.1424)
    beta1 = 0.5443 * (1 + 0.2470 * lit_x)
    beta2 = 2810.36
    beta3 = 5131.9616 * (1 -.2353* b1 * lit_x )
    beta4 = 58.2769 * (1 + 6.7806 * lit_x)
    alpha = 2.5 * (1 + -0.88815489 * lit_x + 0.84602943 * lit_x **2)
    sigma = (1 + np.exp(-(ell - beta3)/beta4))**-1
    poly = (beta1 * (ell/beta2)**alpha -1)
    
    return (poly*sigma + 1).flatten() 

In [45]:
cosmos = [ombs[i] + omcs[i] for i in range(len(ombs))]    #x3 variable
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 2))   #ind variables
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = cosmos[i]  #ommh2

In [46]:
p0 = 0 #initial values

In [47]:
x1 = params[:,0]
x2 = params[:,1]

In [48]:
answers, _ = curve_fit(func,(x1, x2),camb_data_format, p0)  #doing the curve fitting

In [49]:
answers

array([-0.34722361])